In [1]:
import pandas as pd
import scipy.sparse as sparse
import numpy as np
from scipy.sparse.linalg import spsolve


In [2]:
#website_url = 'http://archive.ics.uci.edu/ml/machine-learning-databases/00352/Online%20Retail.xlsx'
retail_data = pd.read_excel('OnlineRetail.xlsx') # This may take a couple minutes

In [3]:
retail_data.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850.0,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.75,17850.0,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom


In [4]:
retail_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 541909 entries, 0 to 541908
Data columns (total 8 columns):
InvoiceNo      541909 non-null object
StockCode      541909 non-null object
Description    540455 non-null object
Quantity       541909 non-null int64
InvoiceDate    541909 non-null datetime64[ns]
UnitPrice      541909 non-null float64
CustomerID     406829 non-null float64
Country        541909 non-null object
dtypes: datetime64[ns](1), float64(2), int64(1), object(4)
memory usage: 33.1+ MB


In [5]:
cleaned_retail = retail_data.loc[pd.isnull(retail_data.CustomerID) == False]

In [6]:
cleaned_retail.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 406829 entries, 0 to 541908
Data columns (total 8 columns):
InvoiceNo      406829 non-null object
StockCode      406829 non-null object
Description    406829 non-null object
Quantity       406829 non-null int64
InvoiceDate    406829 non-null datetime64[ns]
UnitPrice      406829 non-null float64
CustomerID     406829 non-null float64
Country        406829 non-null object
dtypes: datetime64[ns](1), float64(2), int64(1), object(4)
memory usage: 27.9+ MB


In [7]:
item_lookup = cleaned_retail[['StockCode', 'Description']].drop_duplicates() # Only get unique item/description pairs
item_lookup['StockCode'] = item_lookup.StockCode.astype(str) # Encode as strings for future lookup ease

In [8]:
item_lookup.head()

,StockCode,Description
0,85123A,WHITE HANGING HEART T-LIGHT HOLDER
1,71053,WHITE METAL LANTERN
2,84406B,CREAM CUPID HEARTS COAT HANGER
3,84029G,KNITTED UNION FLAG HOT WATER BOTTLE
4,84029E,RED WOOLLY HOTTIE WHITE HEART.


In [9]:
cleaned_retail.loc['CustomerID'] = cleaned_retail.CustomerID.astype(int) # Convert to int for customer ID

/home/xavier/.local/lib/python3.5/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [10]:
cleaned_retail.query('CustomerID==13672')

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
51024,C540634,85232B,SET OF 3 BABUSHKA STACKING TINS,-3.0,2011-01-10 12:02:00,4.95,13672.0,United Kingdom
51025,C540634,21655,HANGING RIDGE GLASS T-LIGHT HOLDER,-12.0,2011-01-10 12:02:00,1.69,13672.0,United Kingdom
51026,C540634,79323P,PINK CHERRY LIGHTS,-4.0,2011-01-10 12:02:00,6.75,13672.0,United Kingdom
53203,540825,79323P,PINK CHERRY LIGHTS,4.0,2011-01-11 13:54:00,6.75,13672.0,United Kingdom
53204,540825,21655,HANGING RIDGE GLASS T-LIGHT HOLDER,12.0,2011-01-11 13:54:00,1.69,13672.0,United Kingdom
53205,540825,85232B,SET OF 3 BABUSHKA STACKING TINS,3.0,2011-01-11 13:54:00,4.95,13672.0,United Kingdom
88001,C543744,85232B,SET OF 3 BABUSHKA STACKING TINS,-3.0,2011-02-11 13:43:00,4.95,13672.0,United Kingdom
88002,C543744,21655,HANGING RIDGE GLASS T-LIGHT HOLDER,-12.0,2011-02-11 13:43:00,1.69,13672.0,United Kingdom
88003,C543744,79323P,PINK CHERRY LIGHTS,-4.0,2011-02-11 13:43:00,6.75,13672.0,United Kingdom
88004,C543745,79323B,BLACK CHERRY LIGHTS,-4.0,2011-02-11 13:46:00,6.75,13672.0,United Kingdom


In [11]:
grouped_cleaned = cleaned_retail.groupby(['CustomerID', 'StockCode']).sum().reset_index() # Group together

In [12]:
#we will use this later
xgboost_data  = cleaned_retail.groupby(['CustomerID', 'StockCode','Country']).sum().reset_index() # Group together
xgboost_data.loc[xgboost_data.Quantity==0,'Quantity'] = 1 # Replace a sum of zero purchases with a one to
# indicate purchased
xgboost_data = xgboost_data.query('Quantity > 0') # Only get customers where purchase totals were positive

In [13]:
xgboost_data.head()

,CustomerID,StockCode,Country,Quantity,UnitPrice
0,12346.0,23166,United Kingdom,1.0,2.08
1,12347.0,16008,Iceland,24.0,0.25
2,12347.0,17021,Iceland,36.0,0.30
3,12347.0,20665,Iceland,6.0,2.95
4,12347.0,20719,Iceland,40.0,3.40


In [14]:
print(xgboost_data.query('CustomerID==13672'))

       CustomerID StockCode         Country  Quantity  UnitPrice
59631     13672.0     21655  United Kingdom       1.0       8.45
59632     13672.0    79323B  United Kingdom       4.0      13.50
59634     13672.0    85232B  United Kingdom       1.0      24.75


In [15]:
grouped_cleaned.head()

,CustomerID,StockCode,Quantity,UnitPrice
0,12346.0,23166,0.0,2.08
1,12347.0,16008,24.0,0.25
2,12347.0,17021,36.0,0.30
3,12347.0,20665,6.0,2.95
4,12347.0,20719,40.0,3.40


In [16]:
grouped_cleaned.loc[grouped_cleaned.Quantity==0,'Quantity'] = 1 # Replace a sum of zero purchases with a one to
# indicate purchased

In [17]:
grouped_purchased = grouped_cleaned.query('Quantity > 0') # Only get customers where purchase totals were positive

In [18]:
grouped_purchased.head()

,CustomerID,StockCode,Quantity,UnitPrice
0,12346.0,23166,1.0,2.08
1,12347.0,16008,24.0,0.25
2,12347.0,17021,36.0,0.30
3,12347.0,20665,6.0,2.95
4,12347.0,20719,40.0,3.40


In [19]:
grouped_cleaned.head()

,CustomerID,StockCode,Quantity,UnitPrice
0,12346.0,23166,1.0,2.08
1,12347.0,16008,24.0,0.25
2,12347.0,17021,36.0,0.30
3,12347.0,20665,6.0,2.95
4,12347.0,20719,40.0,3.40


In [20]:
grouped_purchased = grouped_purchased[['StockCode', 'Quantity', 'CustomerID']] # Get rid of unnecessary info

In [21]:
grouped_purchased.query('CustomerID==13672')

,StockCode,Quantity,CustomerID
59621,21655,1.0,13672.0
59622,79323B,4.0,13672.0
59624,85232B,1.0,13672.0


In [22]:
customers = list(np.sort(grouped_purchased.CustomerID.unique())) # Get our unique customers
products = list(grouped_purchased.StockCode.unique()) # Get our unique products that were purchased
quantity = list(grouped_purchased.Quantity) # All of our purchases

rows = grouped_purchased.CustomerID.astype('category', categories = customers).cat.codes 
# Get the associated row indices
cols = grouped_purchased.StockCode.astype('category', categories = products).cat.codes 
# Get the associated column indices
purchases_sparse = sparse.csr_matrix((quantity, (rows, cols)), shape=(len(customers), len(products)))

In [23]:
purchases_sparse

<4338x3664 sparse matrix of type '<class 'numpy.float64'>'
	with 266723 stored elements in Compressed Sparse Row format>

In [24]:
matrix_size = purchases_sparse.shape[0]*purchases_sparse.shape[1] # Number of possible interactions in the matrix
num_purchases = len(purchases_sparse.nonzero()[0]) # Number of items interacted with
sparsity = 100*(1 - (num_purchases/matrix_size))
sparsity

98.32190920694744

In [25]:
import random

In [26]:
def make_train(purchases, pct_test = 0.2):
    '''
    This function will take in the original user-item matrix and "mask" a percentage of the original ratings where a
    user-item interaction has taken place for use as a test set. The test set will contain all of the original ratings, 
    while the training set replaces the specified percentage of them with a zero in the original ratings matrix. 
    
    parameters: 
    
    purchases - the original purchases matrix from which you want to generate a train/test set. Test is just a complete
    copy of the original set. This is in the form of a sparse csr_matrix. 
    
    pct_test - The percentage of user-item interactions where an interaction took place that you want to mask in the 
    training set for later comparison to the test set, which contains all of the original ratings. 
    
    returns:
    
    training_set - The altered version of the original data with a certain percentage of the user-item pairs 
    that originally had interaction set back to zero.
    
    test_set - A copy of the original ratings matrix, unaltered, so it can be used to see how the rank order 
    compares with the actual interactions.
    
    user_inds - From the randomly selected user-item indices, which user rows were altered in the training data.
    This will be necessary later when evaluating the performance via AUC.
    '''
    test_set = purchases.copy() # Make a copy of the original set to be the test set. 
    test_set[test_set != 0] = 1 # Store the test set as a binary preference matrix
    training_set = purchases.copy() # Make a copy of the original data we can alter as our training set. 
    nonzero_inds = training_set.nonzero() # Find the indices in the ratings data where an interaction exists
    nonzero_pairs = list(zip(nonzero_inds[0], nonzero_inds[1])) # Zip these pairs together of user,item index into list
    random.seed(0) # Set the random seed to zero for reproducibility
    num_samples = int(np.ceil(pct_test*len(nonzero_pairs))) # Round the number of samples needed to the nearest integer
    samples = random.sample(nonzero_pairs, num_samples) # Sample a random number of user-item pairs without replacement
    user_inds = [index[0] for index in samples] # Get the user row indices
    item_inds = [index[1] for index in samples] # Get the item column indices
    training_set[user_inds, item_inds] = 0 # Assign all of the randomly chosen user-item pairs to zero
    training_set.eliminate_zeros() # Get rid of zeros in sparse array storage after update to save space
    return training_set, test_set, list(set(user_inds)) # Output the unique list of user rows that were altered  

In [27]:
product_train, product_test, product_users_altered = make_train(purchases_sparse, pct_test = 0.2)

In [28]:
np.shape(product_users_altered)

(4042,)

In [29]:
import implicit
from implicit import alternating_least_squares

In [30]:
%env OPENBLAS_NUM_THREADS=1

alpha = 15
#user_vecs, item_vecs = alternating_least_squares((product_train*alpha).astype('double'), 
                                                          #factors=20, 
                                                          #regularization = 0.1, 
                                                         #iterations = 50)

#self.user_factors[userid]
            
# initialize a model
model = implicit.als.AlternatingLeastSquares(factors=20)

#sparse matrix of item, user with alpha multiplication
item_user_data = (product_train*alpha).astype('double').T
print("item_user_data")
print(np.shape(item_user_data))

# train the model on a sparse matrix of item/user/confidence weights
model.fit(item_user_data)


# recommend items for a user
userid = 1
user_items = item_user_data.T.tocsr()
recommendations = model.recommend(userid, user_items)
user_vecs = model.user_factors
item_vecs = model.item_factors

env: OPENBLAS_NUM_THREADS=1
item_user_data
(3664, 4338)


In [31]:
from sklearn import metrics

In [32]:
def auc_score(predictions, test):
    '''
    This simple function will output the area under the curve using sklearn's metrics. 
    
    parameters:
    
    - predictions: your prediction output
    
    - test: the actual target result you are comparing to
    
    returns:
    
    - AUC (area under the Receiver Operating Characterisic curve)
    '''
    fpr, tpr, thresholds = metrics.roc_curve(test, predictions)
    return metrics.auc(fpr, tpr)

In [33]:
def calc_mean_auc(training_set, altered_users, predictions, test_set):
    '''
    This function will calculate the mean AUC by user for any user that had their user-item matrix altered. 
    
    parameters:
    
    training_set - The training set resulting from make_train, where a certain percentage of the original
    user/item interactions are reset to zero to hide them from the model 
    
    predictions - The matrix of your predicted ratings for each user/item pair as output from the implicit MF.
    These should be stored in a list, with user vectors as item zero and item vectors as item one. 
    
    altered_users - The indices of the users where at least one user/item pair was altered from make_train function
    
    test_set - The test set constucted earlier from make_train function
    
    
    
    returns:
    
    The mean AUC (area under the Receiver Operator Characteristic curve) of the test set only on user-item interactions
    there were originally zero to test ranking ability in addition to the most popular items as a benchmark.
    '''
    
    
    store_auc = [] # An empty list to store the AUC for each user that had an item removed from the training set
    popularity_auc = [] # To store popular AUC scores
    pop_items = np.array(test_set.sum(axis = 0)).reshape(-1) # Get sum of item iteractions to find most popular
    item_vecs = predictions[1]
    for user in altered_users: # Iterate through each user that had an item altered
        training_row = training_set[user,:].toarray().reshape(-1) # Get the training set row
        zero_inds = np.where(training_row == 0) # Find where the interaction had not yet occurred
        # Get the predicted values based on our user/item vectors
        user_vec = predictions[0][user,:]
        pred = user_vec.dot(item_vecs).toarray()[0,zero_inds].reshape(-1)
        # Get only the items that were originally zero
        # Select all ratings from the MF prediction for this user that originally had no iteraction
        actual = test_set[user,:].toarray()[0,zero_inds].reshape(-1) 
        # Select the binarized yes/no interaction pairs from the original full data
        # that align with the same pairs in training 
        pop = pop_items[zero_inds] # Get the item popularity for our chosen items
        store_auc.append(auc_score(pred, actual)) # Calculate AUC for the given user and store
        popularity_auc.append(auc_score(pop, actual)) # Calculate AUC using most popular and score
    # End users iteration
    
    return float('%.3f'%np.mean(store_auc)), float('%.3f'%np.mean(popularity_auc))  
   # Return the mean AUC rounded to three decimal places for both test and popularity benchmark

In [34]:
np.shape(product_users_altered)

(4042,)

In [35]:
calc_mean_auc(product_train, product_users_altered, 
              [sparse.csr_matrix(user_vecs), sparse.csr_matrix(item_vecs.T)], product_test)
# AUC for our recommender system

(0.868, 0.814)

In [36]:
customers_arr = np.array(customers) # Array of customer IDs from the ratings matrix
products_arr = np.array(products) # Array of product IDs from the ratings matrix

In [37]:
products_arr

array(['23166', '16008', '17021', ..., '84613C', '84206B', '21414'],
      dtype='<U21')

In [38]:
def get_items_purchased(customer_id, mf_train, customers_list, products_list, item_lookup):
    '''
    This just tells me which items have been already purchased by a specific user in the training set. 
    
    parameters: 
    
    customer_id - Input the customer's id number that you want to see prior purchases of at least once
    
    mf_train - The initial ratings training set used (without weights applied)
    
    customers_list - The array of customers used in the ratings matrix
    
    products_list - The array of products used in the ratings matrix
    
    item_lookup - A simple pandas dataframe of the unique product ID/product descriptions available
    
    returns:
    
    A list of item IDs and item descriptions for a particular customer that were already purchased in the training set
    '''
    cust_ind = np.where(customers_list == customer_id)[0][0] # Returns the index row of our customer id
    purchased_ind = mf_train[cust_ind,:].nonzero()[1] # Get column indices of purchased items
    prod_codes = products_list[purchased_ind] # Get the stock codes for our purchased items
    return item_lookup.loc[item_lookup.StockCode.isin(prod_codes)]

In [39]:
customers_arr[:5]

array([ 12346.,  12347.,  12348.,  12349.,  12350.])

In [40]:
get_items_purchased(12346, product_train, customers_arr, products_arr, item_lookup)

,StockCode,Description
61619,23166,MEDIUM CERAMIC TOP STORAGE JAR


In [41]:
from sklearn.preprocessing import MinMaxScaler

In [42]:
def rec_items(customer_id, mf_train, user_vecs, item_vecs, customer_list, item_list, item_lookup, num_items = 10):
    '''
    This function will return the top recommended items to our users 
    
    parameters:
    
    customer_id - Input the customer's id number that you want to get recommendations for
    
    mf_train - The training matrix you used for matrix factorization fitting
    
    user_vecs - the user vectors from your fitted matrix factorization
    
    item_vecs - the item vectors from your fitted matrix factorization
    
    customer_list - an array of the customer's ID numbers that make up the rows of your ratings matrix 
                    (in order of matrix)
    
    item_list - an array of the products that make up the columns of your ratings matrix
                    (in order of matrix)
    
    item_lookup - A simple pandas dataframe of the unique product ID/product descriptions available
    
    num_items - The number of items you want to recommend in order of best recommendations. Default is 10. 
    
    returns:
    
    - The top n recommendations chosen based on the user/item vectors for items never interacted with/purchased
    '''
    
    cust_ind = np.where(customer_list == customer_id)[0][0] # Returns the index row of our customer id
    pref_vec = mf_train[cust_ind,:].toarray() # Get the ratings from the training set ratings matrix
    pref_vec = pref_vec.reshape(-1) + 1 # Add 1 to everything, so that items not purchased yet become equal to 1
    pref_vec[pref_vec > 1] = 0 # Make everything already purchased zero
    rec_vector = user_vecs[cust_ind,:].dot(item_vecs.T) # Get dot product of user vector and all item vectors
    # Scale this recommendation vector between 0 and 1
    min_max = MinMaxScaler()
    rec_vector_scaled = min_max.fit_transform(rec_vector.reshape(-1,1))[:,0] 
    recommend_vector = pref_vec*rec_vector_scaled 
    # Items already purchased have their recommendation multiplied by zero
    product_idx = np.argsort(recommend_vector)[::-1][:num_items] # Sort the indices of the items into order 
    # of best recommendations
    rec_list = [] # start empty list to store items
    for index in product_idx:
        code = item_list[index]
        rec_list.append([code, item_lookup.Description.loc[item_lookup.StockCode == code].iloc[0]]) 
        # Append our descriptions to the list
    codes = [item[0] for item in rec_list]
    descriptions = [item[1] for item in rec_list]
    final_frame = pd.DataFrame({'StockCode': codes, 'Description': descriptions}) # Create a dataframe 
    return final_frame[['StockCode', 'Description']] # Switch order of columns around


In [43]:
rec_items(12346, product_train, user_vecs, item_vecs, customers_arr, products_arr, item_lookup,
                       num_items = 10)

,StockCode,Description
0,23165,LARGE CERAMIC TOP STORAGE JAR
1,23167,SMALL CERAMIC TOP STORAGE JAR
2,23294,SET OF 6 SNACK LOAF BAKING CASES
3,23296,SET OF 6 TEA TIME BAKING CASES
4,22963,JAM JAR WITH GREEN LID
5,23189,SET OF 12 FORK CANDLES
6,23545,WRAP RED DOILEY
7,23295,SET OF 12 MINI LOAF BAKING CASES
8,23293,SET OF 12 FAIRY CAKE BAKING CASES
9,23236,DOILEY STORAGE TIN


In [44]:
get_items_purchased(12353, product_train, customers_arr, products_arr, item_lookup)

,StockCode,Description
2148,37446,MINI CAKE STAND WITH HANGING CAKES
2149,37449,CERAMIC CAKE STAND + HANGING CAKES
4859,37450,CERAMIC CAKE BOWL + HANGING CAKES
5108,22890,NOVELTY BISCUITS CAKE STAND 3 TIER


In [45]:
rec_items(12353, product_train, user_vecs, item_vecs, customers_arr, products_arr, item_lookup,
                       num_items = 10)

,StockCode,Description
0,22055,MINI CAKE STAND HANGING STRAWBERY
1,37447,CERAMIC CAKE DESIGN SPOTTED PLATE
2,37448,CERAMIC CAKE DESIGN SPOTTED MUG
3,22059,CERAMIC STRAWBERRY DESIGN MUG
4,22649,STRAWBERRY FAIRY CAKE TEAPOT
5,22057,CERAMIC PLATE STRAWBERRY DESIGN
6,22644,CERAMIC CHERRY CAKE MONEY BANK
7,22063,CERAMIC BOWL WITH STRAWBERRY DESIGN
8,22066,LOVE HEART TRINKET POT
9,22647,CERAMIC LOVE HEART MONEY BANK


In [46]:
get_items_purchased(12361, product_train, customers_arr, products_arr, item_lookup)

,StockCode,Description
34,22326,ROUND SNACK BOXES SET OF4 WOODLAND
35,22629,SPACEBOY LUNCH BOX
37,22631,CIRCUS PARADE LUNCH BOX
93,20725,LUNCH BAG RED RETROSPOT
369,22382,LUNCH BAG SPACEBOY DESIGN
547,22328,ROUND SNACK BOXES SET OF 4 FRUITS
549,22630,DOLLY GIRL LUNCH BOX
1241,22555,PLASTERS IN TIN STRONGMAN
58132,20725,LUNCH BAG RED SPOTTY


In [47]:
rec_items(12361, product_train, user_vecs, item_vecs, customers_arr, products_arr, item_lookup,
                       num_items = 10)

,StockCode,Description
0,20726,LUNCH BAG WOODLAND
1,20727,LUNCH BAG BLACK SKULL.
2,23206,LUNCH BAG APPLE DESIGN
3,22662,LUNCH BAG DOLLY GIRL DESIGN
4,20728,LUNCH BAG CARS BLUE
5,23207,LUNCH BAG ALPHABET DESIGN
6,20724,RED RETROSPOT CHARLOTTE BAG
7,22383,LUNCH BAG SUKI DESIGN
8,22356,CHARLOTTE BAG PINK POLKADOT
9,23209,LUNCH BAG DOILEY PATTERN


In [48]:
def prediction_matrix(mf_train, user_vecs, item_vecs, customer_list, item_list, item_lookup):
    '''
    This function will return the normalized prediction for each item and each user:
    
    customer_id - Input the customer's id number that you want to get recommendations for
    
    mf_train - The training matrix you used for matrix factorization fitting
    
    user_vecs - the user vectors from your fitted matrix factorization
    
    item_vecs - the item vectors from your fitted matrix factorization
    
    customer_list - an array of the customer's ID numbers that make up the rows of your ratings matrix 
                    (in order of matrix)
    
    item_list - an array of the products that make up the columns of your ratings matrix
                    (in order of matrix)
    
    item_lookup - A simple pandas dataframe of the unique product ID/product descriptions available
    
  
    returns:
    
    - The full matrix of predictions for all users and items
    '''
    #item_factors_matrix = (item_vecs * np.shape(user_vecs)[0]).T  
    item_factors_matrix = item_vecs.T  
    rec_matrix = user_vecs.dot(item_factors_matrix) # Get dot product of user vector and all item vectors
    # Scale this recommendation vector between 0 and 1
    min_max = MinMaxScaler()
    min_max.fit(rec_matrix)
    rec_matrix_scaled = min_max.transform(rec_matrix)
    return rec_matrix_scaled

In [49]:
output_predictions = prediction_matrix(product_train, user_vecs, item_vecs, customers_arr, products_arr, item_lookup)

## XGBOOST Ensemble ##

In [50]:
print(xgboost_data.query('CustomerID==13672'))

       CustomerID StockCode         Country  Quantity  UnitPrice
59631     13672.0     21655  United Kingdom       1.0       8.45
59632     13672.0    79323B  United Kingdom       4.0      13.50
59634     13672.0    85232B  United Kingdom       1.0      24.75


In [51]:
xgboost_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 266733 entries, 0 to 267624
Data columns (total 5 columns):
CustomerID    266733 non-null float64
StockCode     266733 non-null object
Country       266733 non-null object
Quantity      266733 non-null float64
UnitPrice     266733 non-null float64
dtypes: float64(3), object(2)
memory usage: 12.2+ MB


In [52]:
#there is something wrong with this user
#xgboost_data = xgboost_data.query('CustomerID!=15802')

In [53]:
#TODO: Turn country into a numerical variable, remove other columns from retail_data except unit price, add a column for ALS value, add column for target, set all to 1. Introduce ALS value from result matrix. Add random negatives by replicating every row, changing to a random user ID and getting their ALS  value for that item (ideally we would change item, but we'd need to know the unit price, which is not immediate)


In [54]:
xgboost_data.head()

,CustomerID,StockCode,Country,Quantity,UnitPrice
0,12346.0,23166,United Kingdom,1.0,2.08
1,12347.0,16008,Iceland,24.0,0.25
2,12347.0,17021,Iceland,36.0,0.30
3,12347.0,20665,Iceland,6.0,2.95
4,12347.0,20719,Iceland,40.0,3.40


In [55]:
#Let's add a column for the value of ALS, and one for the positive negative label
xgboost_data = xgboost_data.assign(ALS=0., label=1)

In [56]:
xgboost_data.head()

,CustomerID,StockCode,Country,Quantity,UnitPrice,ALS,label
0,12346.0,23166,United Kingdom,1.0,2.08,0.0,1
1,12347.0,16008,Iceland,24.0,0.25,0.0,1
2,12347.0,17021,Iceland,36.0,0.30,0.0,1
3,12347.0,20665,Iceland,6.0,2.95,0.0,1
4,12347.0,20719,Iceland,40.0,3.40,0.0,1


In [57]:
np.shape(xgboost_data)

(266733, 7)

In [58]:
#now let's prepare a matrix of random negatives
xgboost_negatives = xgboost_data.copy()
random_list = np.random.choice(products_arr,266733)
xgboost_negatives['StockCode'] = random_list
xgboost_negatives['label'] =0

In [59]:
xgboost_negatives.head()

,CustomerID,StockCode,Country,Quantity,UnitPrice,ALS,label
0,12346.0,90027D,United Kingdom,1.0,2.08,0.0,0
1,12347.0,21785,Iceland,24.0,0.25,0.0,0
2,12347.0,22765,Iceland,36.0,0.30,0.0,0
3,12347.0,35648,Iceland,6.0,2.95,0.0,0
4,12347.0,22275,Iceland,40.0,3.40,0.0,0


In [60]:
xgboost_data.head()

,CustomerID,StockCode,Country,Quantity,UnitPrice,ALS,label
0,12346.0,23166,United Kingdom,1.0,2.08,0.0,1
1,12347.0,16008,Iceland,24.0,0.25,0.0,1
2,12347.0,17021,Iceland,36.0,0.30,0.0,1
3,12347.0,20665,Iceland,6.0,2.95,0.0,1
4,12347.0,20719,Iceland,40.0,3.40,0.0,1


In [61]:
frames = [xgboost_data,xgboost_negatives]
xgboost_final_data = pd.concat(frames)

In [62]:
xgboost_final_data.head()

,CustomerID,StockCode,Country,Quantity,UnitPrice,ALS,label
0,12346.0,23166,United Kingdom,1.0,2.08,0.0,1
1,12347.0,16008,Iceland,24.0,0.25,0.0,1
2,12347.0,17021,Iceland,36.0,0.30,0.0,1
3,12347.0,20665,Iceland,6.0,2.95,0.0,1
4,12347.0,20719,Iceland,40.0,3.40,0.0,1


In [63]:
xgboost_final_data.tail()

,CustomerID,StockCode,Country,Quantity,UnitPrice,ALS,label
267620,18287.0,22280,United Kingdom,24.0,2.10,0.0,0
267621,18287.0,35265,United Kingdom,6.0,2.55,0.0,0
267622,18287.0,23192,United Kingdom,96.0,2.90,0.0,0
267623,18287.0,47594A,United Kingdom,120.0,4.55,0.0,0
267624,18287.0,21331,United Kingdom,48.0,3.30,0.0,0


In [64]:
users_index = pd.DataFrame({'User':customers_arr,'Pos':np.arange(customers_arr.size)}, index=customers_arr)
items_index = pd.DataFrame({'Item':products_arr,'Pos':np.arange(products_arr.size)}, index=products_arr)

In [65]:
def set_ALS_values_fast(data,customers,products,predictions):
    x=np.array(customers.Pos[data.CustomerID]).astype(int)
    y=np.array(products.Pos[data.StockCode.astype(str)]).astype(int)
    data.ALS = predictions[x,y]    

In [66]:
customers_inxgboostdata = np.array(users_index.Pos[xgboost_data.CustomerID]).astype(int)
items_inxgboostdata = np.array(items_index.Pos[xgboost_data.StockCode]).astype(int)

In [67]:
items_index.min()

Item    10002.0
Pos         0.0
dtype: float64

In [68]:
set_ALS_values_fast(xgboost_final_data, users_index, items_index, output_predictions)

In [69]:
xgboost_final_data.tail()

,CustomerID,StockCode,Country,Quantity,UnitPrice,ALS,label
267620,18287.0,22280,United Kingdom,24.0,2.10,0.871950,0
267621,18287.0,35265,United Kingdom,6.0,2.55,0.528457,0
267622,18287.0,23192,United Kingdom,96.0,2.90,0.590309,0
267623,18287.0,47594A,United Kingdom,120.0,4.55,0.657896,0
267624,18287.0,21331,United Kingdom,48.0,3.30,0.231389,0


In [70]:
np.shape(xgboost_final_data)

(533466, 7)

In [71]:
xgboost_final_data.to_csv('Xgboostdata_with_neagatives.csv')

In [72]:
xgboost_final_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 533466 entries, 0 to 267624
Data columns (total 7 columns):
CustomerID    533466 non-null float64
StockCode     533466 non-null object
Country       533466 non-null object
Quantity      533466 non-null float64
UnitPrice     533466 non-null float64
ALS           533466 non-null float64
label         533466 non-null int64
dtypes: float64(4), int64(1), object(2)
memory usage: 52.6+ MB


We need to convert all features to numerical. We will use categorical encoding

In [73]:
for feature in xgboost_final_data.columns: # Loop through all columns in the dataframe
    if xgboost_final_data[feature].dtype == 'object': # Only apply for columns with categorical strings
        xgboost_final_data[feature] = pd.Categorical(xgboost_final_data[feature]).codes # Replace strings with an integer

In [74]:
xgboost_final_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 533466 entries, 0 to 267624
Data columns (total 7 columns):
CustomerID    533466 non-null float64
StockCode     533466 non-null int16
Country       533466 non-null int8
Quantity      533466 non-null float64
UnitPrice     533466 non-null float64
ALS           533466 non-null float64
label         533466 non-null int64
dtypes: float64(4), int16(1), int64(1), int8(1)
memory usage: 45.9 MB


In [75]:
xgboost_final_data.tail()

,CustomerID,StockCode,Country,Quantity,UnitPrice,ALS,label
267620,18287.0,3944,35,24.0,2.10,0.871950,0
267621,18287.0,5194,35,6.0,2.55,0.528457,0
267622,18287.0,4808,35,96.0,2.90,0.590309,0
267623,18287.0,5396,35,120.0,4.55,0.657896,0
267624,18287.0,3281,35,48.0,3.30,0.231389,0


We will now split into train and test

In [76]:
#first we need to shuffle rows since we have now all positive and negatives together
xgboost_final_data = xgboost_final_data.sample(frac=1).reset_index(drop=True)
xgboost_final_data = xgboost_final_data.sample(frac=1).reset_index(drop=True)

In [77]:
xgboost_final_data.tail()

,CustomerID,StockCode,Country,Quantity,UnitPrice,ALS,label
533461,17669.0,2315,35,20.0,1.65,0.744559,1
533462,14755.0,1628,35,4.0,4.95,0.805663,1
533463,14286.0,4535,35,6.0,2.95,0.796559,0
533464,13975.0,1732,35,12.0,1.45,0.863837,1
533465,14056.0,3395,35,7.0,40.65,0.795120,0


In [78]:
xgboost_final_data.head()

,CustomerID,StockCode,Country,Quantity,UnitPrice,ALS,label
0,17175.0,3840,35,12.0,0.85,0.538145,0
1,13667.0,6017,35,1.0,2.95,0.616393,1
2,17050.0,6362,35,2.0,8.50,0.637954,0
3,16595.0,1213,35,12.0,0.85,0.812167,1
4,15974.0,5171,35,6.0,3.75,0.794355,0


In [79]:
#now we can select 20% of the data for testing
train_samples = int(len(xgboost_final_data)*0.8)
print (train_samples)
xgboost_train = xgboost_final_data[:train_samples].copy() 
xgboost_test = xgboost_final_data[train_samples:].copy()


426772


In [80]:
#We also need to separate target variable label from the datasets
label_train = xgboost_train.pop('label')
label_test = xgboost_test.pop('label')

In [81]:
xgboost_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 426772 entries, 0 to 426771
Data columns (total 6 columns):
CustomerID    426772 non-null float64
StockCode     426772 non-null int16
Country       426772 non-null int8
Quantity      426772 non-null float64
UnitPrice     426772 non-null float64
ALS           426772 non-null float64
dtypes: float64(4), int16(1), int8(1)
memory usage: 14.2 MB


In [82]:
xgboost_test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 106694 entries, 426772 to 533465
Data columns (total 6 columns):
CustomerID    106694 non-null float64
StockCode     106694 non-null int16
Country       106694 non-null int8
Quantity      106694 non-null float64
UnitPrice     106694 non-null float64
ALS           106694 non-null float64
dtypes: float64(4), int16(1), int8(1)
memory usage: 3.6 MB


In [83]:
xgboost_test.head()

,CustomerID,StockCode,Country,Quantity,UnitPrice,ALS
426772,14410.0,1746,35,1.0,2.50,0.657909
426773,16843.0,3833,35,72.0,9.75,0.733483
426774,16903.0,2326,35,24.0,0.84,0.708083
426775,16637.0,4919,35,3.0,71.90,0.384575
426776,14947.0,669,35,8.0,2.55,0.708686


In [84]:
import xgboost as xgb
from sklearn.model_selection import GridSearchCV

/usr/local/lib/python3.5/dist-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [85]:
xgboost_train.to_csv('xgboost_train.csv')

In [86]:
label_train.to_csv('label_train.csv')

In [ ]:
#Look for more info on the XGBoost parameters here: https://github.com/dmlc/xgboost/blob/master/doc/parameter.md
cv_params = {'max_depth': [3,5,7], 'min_child_weight': [1,3,5]}

ind_params = {'learning_rate': 0.1, 'n_estimators': 1000, 'seed':0, 'subsample': 0.8, 'colsample_bytree': 0.8, 
            'objective': 'binary:logistic'}
optimized_XGB = GridSearchCV(xgb.XGBClassifier(**ind_params), 
                            cv_params, 
                             scoring = 'roc_auc', cv = 5, n_jobs = 1, verbose=2) 
#We optimize for AUC to compare with previous results

In [ ]:
#train XGB
optimized_XGB.fit(xgboost_train, label_train,eval_metric='auc')

In [ ]:
#train error
optimized_XGB.grid_scores_

In [ ]:
#we take our best model given cross validation results
best_params = {'eta': 0.1, 'seed':0, 'subsample': 0.8, 'colsample_bytree': 0.8, 
             'objective': 'binary:logistic', 'max_depth':5, 'min_child_weight':1} 

train_xgb_matrix = xgb.DMatrix(xgboost_train, label_train) # Create our DMatrix to make XGBoost more efficient
best_model = xgb.train(best_params, train_xgb_matrix, num_boost_round = 432)

In [ ]:
#now let's check error on test set
test_xgb_matrix = xgb.DMatrix(xgboost_test)
from sklearn.metrics import roc_auc_score
label_pred = best_model.predict(test_xgb_matrix)
label_pred

In [ ]:
#need to convert values to binary label
label_pred[label_pred > 0.5] = 1
label_pred[label_pred <= 0.5] = 0

In [ ]:
roc_auc_score(label_pred.astype(int), label_test.astype(int))

# Bonus Follow ups

* Improve overall auc by adding more features to xgboost ensemble
* Fine-tune xgboost ensemble by optimizing other hyperparameters (e.g. learning rate)
* Analyze how hyperparameters behave to metrics different from AUC